In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import copy
import itertools
from collections import OrderedDict
import yaml
import re
from yaml import Loader
import functools
import torch

from seq2seq.utils import generate_seed
from seq2seq.data.tokenizer import Tokenizer
from seq2seq.data.dataset import ParallelDataset, LazyParallelDataset,  TextDataset
from seq2seq.models.gnmt import GNMT
from seq2seq.train.loss import LabelSmoothing
from seq2seq.data import config as seq2seq_config
from seq2seq.inference.translator import Translator
from seq2seq.train.trainer import Seq2SeqTrainer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
def load_config(config_filename):
    with open(config_filename) as f:
        config = yaml.load(f, Loader)
    
    def fix_config(sub_config, config):
        for k, val in sub_config.items():
            if isinstance(val, str) and "${" in val:
                substitutes = []
                for re_key in re.findall(r"(\$\{.+?\})", val):
                    substitutes.append(re_key)
                    re_keys = re.search(r"\$\{(.+?)\}", re_key).groups()
                    for key in re_keys:
                        subst_val = config
                        for name in key.split("@@@"):
                            subst_val = subst_val[name]
                        sub_config[k] = sub_config[k].replace(re_key, subst_val)
            elif isinstance(val, dict):
                fix_config(val, config)
    fix_config(config, config)
    os.makedirs(config['checkpoint']['save_dir'], exist_ok=True)
    
    return config

config = load_config("./config.yaml")        

In [58]:
DL_PATH = os.environ.get("DL_PATH")
lang = config["setup"]["lang"]
src_lang = lang['src']
tgt_lang = lang["tgt"]
SAVEPATH = os.path.join(DL_PATH, "nmt", f"{src_lang}-{tgt_lang}", "data" )
device =  torch.device("cpu")
# TODO
pad_vocab = 8

vocab_fname = config["setup"]["dataset"]['vocab']
bpe_fname = config["setup"]["dataset"]['bpe']
tokenizer = Tokenizer(vocab_fname=vocab_fname, bpe_fname=bpe_fname, lang=lang, pad=pad_vocab)

Building vocabulary /media/mtb/1268324a-8d38-4c4f-9b71-2a4ddc231fe6/dl/nmt/en-fr/data/vocab.bpe.40000
Vocabulary size 40320


In [59]:
train_src_fname = config['setup']['dataset']['src']['train']
train_tgt_fname = config['setup']['dataset']['tgt']['train']
train_max_len = config['training']["train_max_len"]
train_min_len = config['training']["train_min_len"]
# TODO
train_data = LazyParallelDataset(src_fname=train_src_fname,
                             tgt_fname=train_tgt_fname,
                             tokenizer=tokenizer,
                             min_len=train_min_len,
                             max_len=train_max_len,
                             sort=False,
                             max_size=1000
                             )

In [60]:
model_config = dict(config['model']) | {"vocab_size": tokenizer.vocab_size}

In [61]:

train_loader = train_data.get_loader(batch_size=config["training"]['batch_size'],
                                     seeds=[1],
                                     batch_first=config['model']['batch_first'],
                                     shuffle=True, 
                                     batching=config['training']['batching'],
                                     batching_opt={'num_buckets': config['training']['num_buckets']},
                                     num_workers=0,
                                     )

Bucket 0 had length 32
Bucket 1 had length 363
Bucket 2 had length 351
Bucket 3 had length 162
Bucket 4 had length 61
New samples  640


In [62]:
model = GNMT(**model_config).to(device)

In [63]:
train_loader.sampler.set_epoch(0)

In [64]:
d = next(iter(train_loader))

In [65]:
d[0][0].shape

torch.Size([128, 25])

In [66]:
out = model(*d[0], d[1][0] )

torch.Size([128, 25, 1024]) CL 
torch.Size([1, 128, 1024])
torch.Size([1, 128, 1024])
2
4
torch.Size([8, 128, 1024])
torch.Size([128, 25, 1024]) yy 
torch.Size([128]) yy 
torch.Size([8, 128, 1024]) yy 


In [67]:
len(out)

3

In [68]:
out[0].shape

torch.Size([128, 25, 1024])

In [69]:
out[1].shape

torch.Size([128])

In [70]:
out[2].shape

torch.Size([8, 128, 1024])

In [71]:
type(out)

list

In [2]:
test_path = '/media/mtb/1268324a-8d38-4c4f-9b71-2a4ddc231fe6/dl/nmt/en-fr/models/gnmt/eval_epoch_9'
reference_path = '/media/mtb/1268324a-8d38-4c4f-9b71-2a4ddc231fe6/dl/nmt/en-fr/data/test.tok.simplify.fr'

In [10]:
from sacrebleu.metrics.bleu import BLEU
bleu = BLEU(tokenize='intl', effective_order=True)

In [11]:
with open(test_path, 'r') as tfile, open(reference_path, 'r') as rfile:
    for l1, l2 in zip(tfile, rfile):
        print(l1)
        print(l2)
        print(bleu.sentence_score(l1, [l2]).score)

En outre, l'Office note que le Tribunal n'a pas été informé de la décision attaquée.

Les demeurés de UKIP qui refusent ceux qui vivent avec le VIH sont un exemple parfait .

2.3901021968803136
Les de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de

Vous parlez de quand Nigel Farage dit que le NHS ne doit pas servir à payer pour ceux qui viennent au Royaume @-@ Uni faire du tourisme de santé , et quand , avec ça à l&apos; esprit , il répond oui quand l&apos; intervieweur lui demande spécifiquement si les personnes vivant avec le VIH sont inclus dans les indésirables .

0.2850578215211531
En outre, l'